<a href="https://colab.research.google.com/github/DanielDialektico/machine-learning-practices/blob/main/notebooks/Otros/Tutorial_API_de_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

# **Consumo de API de Spotify con Python usando Spotipy 🎼**

# **Introducción**

En este **notebook** pondremos en práctica cómo consumir la **API** de **Spotify** usando la librería Spotipy de Python, donde obtendremos datos de canciones y artistas, y los ordenaremos en formato tabular para finalmente almacenar la información en un CSV.

Este notebook es parte del [tutorial de cómo consumir la API de Spotify con Python](https://dialektico.com/tutorial-api-spotify-python/), donde se pueden hallar instrucciones más detalladas.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/01/APIS_colab.jpg" width="300" /></center>

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/01/APIS_Colab_obj.jpg" width="300" /></center>

<br>

# **Instalación de librerías**

Instalamos la librerías:

***Nota*:** *Las instalaciones se realizan a pesar de que algunas librerías ya están integradas de forma nativa en Colab, esto para asegurar que el Notebook no presente problemas de ejecución si se dan cambios de sintaxis entre versiones de librerías.*

In [ ]:
!pip install pandas==2.2.2
!pip install tqdm==4.67.1
!pip install spotipy==2.25.0

<br>

# **Se añaden las credenciales de autenticación**

Se declaran las variables con las **credenciales** ([ver tutorial](https://dialektico.com/tutorial-api-spotify-python/)).

In [ ]:
# Se importan las librerías necesarias.
import warnings
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from tqdm import tqdm

# Se filtran las advertencias.
warnings.filterwarnings('ignore')

# Se declara el ID del cliente.
client_id = 'Añade tu Client ID'

# Se declara el valor del secret client.
client_secret = 'Añade tu Client Secret'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

<br>

# **Usando Spotipy para extraer datos**

Comenzamos creando el objeto que nos permitirá consumir los endpoints de la API.

In [ ]:
# Se crea el cliente que se utilizará para consumir la API.
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Probamos utilizando la función search, la cual permite obtener información del catálogo de Spotify sobre **álbumes**, **artistas**,** listas de reproducción**, **canciones**, **programas**, **episodios** o **audiolibros** que coincidan con una cadena de palabras clave (es decir, buscará y traerá información sobre algún elemento especificado).

Visualizamos el formato en el que se entrega la información, haciendo una búsqueda de datos de la agrupación **Blackpink**.

In [ ]:
# Se busca información sobre Blackpink.
result = sp.search(q='Blackpink', type='artist', limit=1)

# Se imprime el resultado.
result

Ahora accedemos a un elemento en particular del diccionario, en este caso, al **número total de seguidores** del grupo en la plataforma:

In [ ]:
# Se imprime el número de seguidores de Blackpink.
result['artists']['items'][0]['followers']['total']

Obtenemos el top de 10 canciones más populares de la banda utilizando el método [artist_top_tracks](https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.artist_top_tracks):

In [ ]:
# Se obtiene ID de Blackpink de la información antes extraída.
blackpink_id = result['artists']['items'][0]['id']

# Se obtienen las 10 canciones más escuchadas de Blackpink.
results = sp.artist_top_tracks(blackpink_id)

# Se imprime el nombre de las canciones.
for track in results['tracks']:
    print(track['name'])

<br>

## **Extrayendo datos en tablas**

Creamos una función para la extracción de 50 canciones de un **género** definido (parámetro de entrada), sus características, y ordena la información en un dataFrame de **Pandas**:

In [ ]:
def get_tracks_by_genre(genre):
    """
    Función que obtiene información de 50 canciones de un género musical de
    Spotify y organiza la información de un Pandas dataframe.
    """

    # Se buscan 50 canciones del género especificado.
    results = sp.search(q=f'genre:{genre}', type='track', limit=50)

    # Se extraen los IDs de las canciones.
    track_ids = [track['id'] for track in results['tracks']['items']]

    # Se obtienen detalles de las canciones.
    tracks_info = sp.tracks(tracks=track_ids)

    # Se crea una lista para almacenar los datos.
    track_data = []

    # Se obtienen los datos de cada canción y se añaden a la lista.
    for track in tracks_info['tracks']:
        track_data.append({
            'genre': genre,
            'name': track['name'],
            'artists_name': ', '.join([artist['name'] for artist in track['artists']]),  # Si hay varios artistas
            'album_name': track['album']['name'],
            'album_release_date': track['album']['release_date'],
            'duration_ms': track['duration_ms'],
            'popularity': track['popularity'],
            'track_number': track['track_number'],
            'uri': track['uri']
        })

    # Se convierte la lista en un DataFrame de pandas.
    df = pd.DataFrame(track_data)

    return df

Probamos la función con el género **Rock**:

In [ ]:
# Ejemplo de uso con el género 'rock'.
genre = "rock"
get_tracks_by_genre(genre)

**Nota**: una lista de los géneros disponibles se puede consultar en [https://everynoise.com/everynoise1d.html](https://everynoise.com/everynoise1d.html)

Ahora creamos un bucle que obtenga estos datos para cada género especificado de una **lista**, y concatenamos las **tablas** resultantes para obtener un conjunto de datos:

In [ ]:
# Se crea una lista con los géneros musicales deseados.
genres_list = ['world-music', 'salsa', 'rock-n-roll', 'reggae', 'reggaeton', 'pop', 'black-metal', 'k-pop', 'hip-hop', 'electronic']

# Se itera sobre esta lista utilizando sus elementos como entradas de la función creada.
tables = []

for item in tqdm(genres_list, desc = 'Creando conjunto de datos:'):
  table = get_tracks_by_genre(item)
  tables.append(table)

# Se concatenan las tablas y se imprime el resultado.
dataset = pd.concat(tables)
dataset

Por último se descarga en formato CSV:

In [ ]:
# Se convierte a formato CSV.
dataset.to_csv('dataset.csv', index=False)

# Se descarga el CSV.
from google.colab import files
files.download('dataset.csv')

O en formato XLS:

In [ ]:
# Se convierte a formato Excel.
dataset.to_excel('dataset.xlsx', index=False)

# Se descarga el Excel.
from google.colab import files
files.download('dataset.xlsx')

No olvides que puedes encontrar más tópicos como este en https://dialektico.com/.

▶ [Regresar al tutorial](https://dialektico.com/tutorial-api-spotify-python/) 📕

<br>

In [ ]:
# Dialektico Machine learning practices © 2024 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License